# 파이썬스러운 객체
* 다룰 내용
    * ```repr(), bytes()``` 등 객체를 다른 방식으로 표현하는 내장 함수
    * 클래스 메서드로 대안 생성자 구현
    * ```format()```내장 함수와 ```str.format()```메서드에서 사용하는 포맷 언어 확장
    * 읽기 전용 접근만 허용하는 속성 제공

In [31]:
import math
from array import array


class Vector2d:
    """
    Fluent Python ~ p. 340 까지 설명 내용을 포함하는 클래스
    본 책 1장 예제의 확장 버전
    """
    typecode= 'd'
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
    
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
        
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        """
        bytes -> Vector2d 변환 메서드
        """
        typecode = chr(octets[0])
        memv = memoryview(octets[1:].cast(typecode))
        return cls(*memv)
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else: 
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

## 객체의 표현
* ```repr()```
    * ```__repr__()``` 구현 필요함
    * 객체를 개발자가 보고자 하는 형태로 표현한 문자열 (python3 유니코드 문자열)로 반환
    * 객체의 공식적인 문자 표현
    * **가능하다면 같은 값으로 객체를 재생산할 수 있는 유효한 파이썬 표현이어야 함**
* ```str()```
    * ```__str()__``` 구현 필요함
    * 객체를 사용자가 보고자 하는 형태로 표현한 문자열 (python3 유니코드 문자열)로 반환
    * 비공식적 또는 출력하기 좋은 형태의 문자 표현
* ```bytes()```
    * ```__bytes__()``` 구현 필요함
    * 객체 바이트 시퀀스로 표현
* ```str.format() /  format()```
    * ```__format__()``` 구현 필요함
    * 특별 포맷 코드 이용해 객체 표현 문자열 (python3 유니코드 문자열)을 반환
* 추가 설명 자료 [출처](https://docs.python.org/3/reference/datamodel.html)

## @classmethod와 @staticmethod
* ```Java```에서의 ```static```
    * ```Java```에서 ```static``` 키워드가 붙은 변수/메소드르 클래스 변수/메소드라고 함
    * 클래스 변수의 경우 공유 변수의 의미를 지님
        * 클래스를 바탕으로 인스턴스를 생성함
        * 클래스 변수는 해당 클래스를 기반으로 하는 모든 인스턴스가 공유할 수 있는 변수가 됨
    * 클래스 변수/메소드는 인스턴스 생성 전에 사용할 수 있는 변수/메소드로써도 사용
    * (참고 사항) JVM 상 static 영역 관련
* ```@classmethod```
    * 첫 번째 인수로 클래스를 받음
* ```@staticmethod```
    * **첫 번째 인수로 클래스를 받지 않음**

In [13]:
# class 변수와 메소드 예시
import random


class RandomStringGenerator:
    """
    https://github.com/leeseungeun/imsure/blob/develop/src/main/java/com/hana/imsure/common/utils/RandomStringGenerator.java

    비밀번호 초기화 등을 위한 random 인증 번호 생성을 위한 코드 Python version
    (자바 코드를 파이썬 코드로 직역한 것과 다름 없으므로 효율적으로 동작하기 위해서는 리팩토링 필요)
    """
    # 해당 클래스에 인스턴스 메소드가 추가되더라도 해당 문자열 리스트는 모든 인스턴스에 공유되어야 하므로 클래스 메소드가 필요
    # 클래스 변수 사용하는 메소드는 클래스 메소드여야 함 (생성 주기 때문, 파이썬은 자바와 같은 생성 주기 같지 않아 유의)
    char_set = [ 
		"0", "1", "2", "3", "4", "5", "6", "7", "8", "9" ,
		"a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z",
		"A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"
    ]
    
    @classmethod
    def generate_random_string(cls, length):
        """
        본 예시에서는 staticemthod 사용 불가
        """
        
        result = ''
        
        for i in range(0, length + 1):
            result += cls.char_set[random.randint(0, len(cls.char_set) - 1)]
            
        return result

In [12]:
# 5자리의 랜덤 문자열 생성 방법, 클래스 메서드로 만듦으로써 랜덤 문자열을 만들 때마다 인스턴스를 만들 필요 없음
RandomStringGenerator.generate_random_string(5)

'IdTmKH'

In [16]:
# staticmethod vs classmethod
class Test:
    @classmethod
    def clsmethod(a, b):
        print(a, b)
        
    @staticmethod
    def statmethod(a, b):
        print(a, b)

In [19]:
Test.clsmethod(1, 2)

TypeError: clsmethod() takes 2 positional arguments but 3 were given

In [20]:
Test.statmethod(1, 2)

1 2


## 포맷된 출력
* format_spec (포맷 명시자)
    * ```format(my_obj, format_spec)```의 두 번째 인수
    * ```str.format()```에 사용된 포맷 문자열 안에 ```{}로 구분한 대체 필드 안에서 콜론 뒤 문자열```
    * 포맷 명시 간이 언어 (Format Specification Mini-Language)
        * ```b```: int 이진수
        * ```x```: int 16진수
        * ```f```: float 고정소수점
        * ```%```: 백분율
        * [more](https://docs.python.org/3/library/string.html#formatspec)
* 클래스에 ```__format__()``` 정의하지 않을 경우 ```str(my_object)``` 반환

In [30]:
## str.format() 외전

# str.format() 많이 사용하는 방식
print('기본 예시 {}, {}, {}'.format(1, 2, 3))

# 인덱스를 활용한 방식
print('인덱스 예시 {2}, {1}, {0}'.format(*[1,2,3]))

# key를 활용한 방식
print('key 예시 {apple}, {banana}, {cat}'.format(**{'apple':3, 'banana':2, 'cat': 1}))

"""
개인적으로 key를 이용한 string format을 자주 사용하는 경우: 
sqlalchemy와 같은 ORM 사용 안 하고 sqlite3 모듈 이용해 sql 실행할 때 
str.format()을 잘 활용하면 기본 SQL문 하나로 여러 테이블을 조작할 수 있음!!!

** ORM은 관계형 데이터베이스와 객체를 mapping해 사용하는 것을 지칭

"""

# 테이블에 데이터를 넣는 기본 SQL문
insert_sql = 'INSERT INTO {table} {columns} VALUES ({pstmt})'

# insert를 수행할 테이블과 컬럼
parameters = {
    'table': 'presentations',
    'columns': ('number', 'name')    
}
parameters['pstmt'] = ('?,' * len(parameters['columns']))[:-1]
print(insert_sql.format(**parameters))

기본 예시 1, 2, 3
인덱스 예시 3, 2, 1
key 예시 3, 2, 1
INSET INTO presentations ('number', 'name') VALUES (?,?)


## 해시 가능한 Vector2d
* 해시 가능하게 만들기 위해서는 ```__hash__(), __eq__()``` 구현해야 함
    * 해시 가능해야 ```set()``` 변환 가능
    * 책 예제에서 Vector2d 불변형으로 만든 후 진행

* 접근 제한자 (Access Modifier)
    * 필요성: 객체 지향 프로그래밍 (Object Oriented Programming, OOP)의 캡슐화 (Encpsulation)에 기인함
        * 캡슐화
            * 데이터에 private 또는 protected 접근 제한자를 붙임으로써 데이터에 대한 직접적인 접근을 막고, 메소드로써 데이터에 접근하도록 함
            * 메소드 (e.g. getter / setter)로써 데이터 접근하도록 할 경우 변수에 값을 할당하기 전 입력 데이터를 검증할 수 있다는 장점이 있음
            * ```@prorperty``` 데코레이터는 getter 메소드 역할 ([출처](https://hamait.tistory.com/827))
            * [출처](https://www.quora.com/Why-should-we-use-access-modifiers-in-Java)
    * 종류
        * private: 해당 클래스에서만 접근 가능
        * protected:해당 클래스와 그를 상속 받은 클래스에서만 접근 가능
        * public: 모든 클래스에서 접근 가능
        * default / package (Java에서 사용되는 접근 제한자): 해당 패키지의 클래스에서만 접근 가능
    * Python에서의 접근 제한자
        * 다른 언어와 달리 Python에서는 public 접근 제한자가 기본
        * protected로 설정하고 싶은 변수 또는 메서드 앞에 _(single underscore)를 붙여 표시
            * 실제 제약되지 않고 일종의 경고 표시로 사용 ([출처](https://stackoverflow.com/questions/797771/python-protected-attributes))
        * private으로 설정하고 싶은 변수 또는 메서드 앞에 \_\_ (double underscore)를 붙여 표시
            * \_\_(double underscore)를 붙이면 이름이 \_classname\_\_해당 속성 또는 메서드 이름으로 변경되어 해당 이름으로 접근할 수 없음
    * 출처:  패스트캠퍼스 컴퓨터사이언스 익스텐션스쿨 2기 Dave Lee 강사님의 파이썬 객체 지향 프로그래밍에 대한 강의

In [32]:
# 재밌는 Python private 변수 예시
# 출처: https://stackoverflow.com/questions/42801071/creating-a-playing-card-class-python

class Card:
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit
        self.__ranks = [None, "Ace", 2, 3, 4, 5, 6, 7, 8, 9, 10, "Jack", "Queen", "King"]
        self.__suits = {"d": "Diamonds",
                   "c": "Clubs",
                   "h": "Hearts",
                   "s": "Spades"}

In [33]:
heart8 = Card(8, 'h')
dir(heart8)

['_Card__ranks',
 '_Card__suits',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'rank',
 'suit']

In [34]:
heart8.ranks

AttributeError: 'Card' object has no attribute 'ranks'

In [36]:
heart8._Card__ranks

[None, 'Ace', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'Jack', 'Queen', 'King']